In [1]:
from chatbot.data_loader import load_data
from chatbot.intent_router import detect_intent
from chatbot.holdings_engine import (
    count_holdings, performance_by_fund
)
from chatbot.trades_engine import count_trades
from chatbot.combined_engine import trade_vs_performance
from chatbot.response_generator import generate_response


d:\assesment\virt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\assesment\chatbot\response_generator.py:51: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [2]:
holdings, trades = load_data()

In [7]:
def chatbot(question: str):
    intent = detect_intent(question)

    # -----------------------
    # COUNT HOLDINGS
    # -----------------------
    if intent == "COUNT_HOLDINGS":
        fund = question.split("for")[-1].strip().lower()
        result = count_holdings(holdings, fund)

        if result <= 0:
            return "Sorry can not find the answer"

        context = f"Total number of holdings for fund '{fund}': {result}"
        return generate_response(context, question)

    # -----------------------
    # FUND PERFORMANCE
    # -----------------------
    if intent == "FUND_PERFORMANCE":
        perf = performance_by_fund(holdings)

        # Deterministic business logic
        positive_perf = perf[perf > 0]

        if positive_perf.empty:
            return "Sorry can not find the answer"

        context = (
            "Funds ranked by yearly Profit and Loss (PL_YTD):\n\n"
            + positive_perf.to_string()
        )
        return generate_response(context, question)

    # -----------------------
    # COUNT TRADES
    # -----------------------
    if intent == "COUNT_TRADES":
        fund = question.split("for")[-1].strip().lower()
        result = count_trades(trades, fund)

        if result <= 0:
            return "Sorry can not find the answer"

        context = f"Total number of trades for fund '{fund}': {result}"
        return generate_response(context, question)

    # -----------------------
    # TRADE VS PERFORMANCE (SAFE CROSS-DATASET)
    # -----------------------
    if intent == "TRADE_VS_PERFORMANCE":
        table = trade_vs_performance(trades, holdings)

        if table.empty:
            return "Sorry can not find the answer"

        context = (
            "Comparison of trade activity and yearly performance by fund:\n\n"
            + table.to_string()
        )
        return generate_response(context, question)

    # -----------------------
    # UNSUPPORTED
    # -----------------------
    return "Sorry can not find the answer"


In [8]:
chatbot("Which funds performed better depending on the yearly Profit and Loss of that fund.")

'The funds performed better in descending order of yearly Profit and Loss (PL_YTD):\n\n1. ytum (7.229903e+06)\n2. npsmf1 (3.043387e+05)\n3. npsmf2 (2.975902e+05)\n4. npsmf3 (2.073081e+05)\n5. coyold 1 (2.105159e+04)\n6. ig corp (2.068600e+00)'